# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np

## 1.  Optimal foraging of individual agent

In [ ]:
# Read constants CSV
config_df = pd.read_csv("config.csv")

# Converts to dictionary
config = dict(zip(config_df["param"], config_df["value"]))

F = float(config["F"])                     # resource energy
r = float(config["r"])                     # number of resources
generations = int(config["generations"])   # number of iterations
c_s = float(config["c_s"])                 # speed constant
c_a = float(config["c_a"])                 # acuity constant

# Read individual agents' data CSV
single_agent_df = pd.read_csv("./simulation_results/_agents_20250604_004745.csv", delimiter=",")

single_agent_df["encounter_rate"] = single_agent_df["resource_encouters"] / generations
single_agent_df["metabolic_cost"] = c_s * single_agent_df["speed"]**2 + c_a * single_agent_df["acuity"]
single_agent_df["net_energy_gain"] = single_agent_df["encounter_rate"] * F - single_agent_df["metabolic_cost"] 

max_net_energy_gain = single_agent_df["net_energy_gain"].max()
maximizers = single_agent_df[single_agent_df["net_energy_gain"] == max_net_energy_gain]

# Plot heatmap
sns.kdeplot(data=single_agent_df, x="speed", y="acuity",
            weights=single_agent_df["net_energy_gain"], fill=True, cmap="viridis")

# Add maximizers (phenotypes with highest ε)
plt.scatter(maximizers["speed"], maximizers["acuity"], 
            color='red', s=100, edgecolor='black', label="max ε")

plt.title("Net Energy Intake ε(s,a)")
plt.xlabel("Speed")
plt.ylabel("Acuity")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compute mean path length between encounters
single_agent_df["L"] = single_agent_df["speed"] / single_agent_df["encounter_rate"]

# Plot 𝓛 vs. a
sns.lineplot(data=single_agent_df, x="acuity", y="L")
plt.xlabel("Acuity (a)")
plt.ylabel("𝓛(a) = s / ν")
plt.title("Mean path length between encounters vs. Acuity")
plt.grid(True)
plt.show()

In [ ]:
a = single_agent_df["acuity"].values    # Acuity rate values
L = single_agent_df["L"].values         # Mean path length between encounters values

# Compute g and g^2
g = 1 / L
g2 = g**2

# 1st derivative of g²
dg2_da = np.gradient(g2, a)

# Plot 1st derivative of g² vs a
plt.plot(a, dg2_da, label="d(g²)/da")
plt.xlabel("Acuity (a)")
plt.ylabel("d(g²)/da")
plt.title("1st derivative of g² vs Acuity")
plt.grid(True)
plt.legend()
plt.show()

## 2. Steady-state of a population using single strategy